In [1]:
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from keras.models import load_model
tf.test.gpu_device_name()
# 查看显卡驱动
!/opt/bin/nvidia-smi

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Tue Dec  5 13:20:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    32W / 250W |    465MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
model_paths = { 'model_dan': '/kaggle/input/model-for-train-in-public/model_dan_testv2_6times_0.5288.h5',
               'model_kyu': '/kaggle/input/model-for-train-in-public/model_kyu_testv2_6times.h5',
               'model_playstyle': '/kaggle/input/model-for-train-in-public/model_playstyle_test.h5' }

In [3]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}


def prepare_input(moves):
    x = np.zeros((19,19,3))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            #x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            #x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,2] = 1
    #x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x


def prepare_input_Zero(moves):
    # 初始化19x19x17的数组
    x = np.zeros((19, 19, 17))
    x_state = np.zeros((19, 19, 2))
    # 历史棋盘状态的索引
    
    history_index = 0
    num = len(moves)
    last_eight_indices = num - 8
    eight_index = moves[last_eight_indices]
    eight_color= eight_index[0]
    for move_num, move in enumerate(moves):
        color = move[0]  
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        # 根据颜色设置对应的历史棋盘层
        if color == 'B':  # 黑子
            x_state[row, column, 0] = 1
        else:  # 白子
            x_state[row, column, 1] = 1
        # 在最后八个移动中，更新历史状态
        if last_eight_indices <= move_num:
            if eight_color == 'B':  # 黑子
                x[:, :, history_index * 2] = x_state[:, :, 0]
                x[:, :, history_index * 2 + 1] = x_state[:, :, 1]
            else:  # 白子
                x[:, :, history_index * 2] = x_state[:, :, 1]
                x[:, :, history_index * 2 + 1] = x_state[:, :, 0]
            history_index += 1
    # 设置当前玩家层
    if moves:
        if eight_color == 'B':  
            x[:, :, -1] = 0  # 白方
        else:
            x[:, :, -1] = 1  # 黑方
    return x


def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

In [4]:

model = load_model(model_paths['model_dan'])
# Load the corresponding dataset
df = open('/kaggle/input/go-competition/29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]


In [5]:
x_testing=[]
for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)

predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_private_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

344/344 [==============================] - 12s 19ms/step


In [6]:
#model = load_model(model_paths['model_dan'])
# Load the corresponding dataset
df = open('/kaggle/input/go-competition/29_Private Testing Dataset_Public and Private Submission Template_v2/29_Private Testing Dataset_v2/dan_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing=[]
for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)

predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_private_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

344/344 [==============================] - 7s 19ms/step


In [7]:

model = load_model(model_paths['model_kyu'])

# Load the corresponding dataset
df = open('/kaggle/input/go-competition/29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/kyu_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]
x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_private_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

355/355 [==============================] - 7s 19ms/step


In [8]:
# Load your own model. Here we use the baseline model
#model = load_model(model_paths['model_kyu'])

# Load the corresponding dataset
df = open('/kaggle/input/go-competition/29_Private Testing Dataset_Public and Private Submission Template_v2/29_Private Testing Dataset_v2/kyu_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]
x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_private_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

355/355 [==============================] - 7s 19ms/step


In [9]:
# Load your own model. Here we use the baseline model

model = load_model(model_paths['model_playstyle'])
# Load the corresponding dataset
df = open('/kaggle/input/go-competition/29_Public Testing Dataset_Public Submission Template_v2/29_Public Testing Dataset_v2/play_style_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

In [10]:
# 遍历所有游戏数据
for idx, game in enumerate(games):
    num  = 8
    moves_list = game.split(',')
    # 计算当前游戏的长度
    current_length = len(moves_list)
    # 计算需要添加的空移动数量
    if current_length < num:
        padding_length = (num - (current_length % num)) %num
        # 获取第一个动作作为padding_value
        padding_value = moves_list[0]
        # 使用列表切片添加空移动
        moves_list = [padding_value] * padding_length + moves_list
        # 更新games中的数据
        games[idx] = ','.join(moves_list)

In [11]:
x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_Zero(moves_list))

x_testing = np.array(x_testing)
x_testing.shape

(9996, 19, 19, 17)

In [12]:

predictions = model.predict(x_testing)
prediction_numbers = np.argmax(predictions, axis=1)

with open('./public_private_submission_template.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

313/313 [==============================] - 13s 41ms/step


In [13]:
# Load your own model. Here we use the baseline model
#model = load_model(model_paths['model_playstyle'])

# Load the corresponding dataset
df = open('/kaggle/input/go-competition/cleaned_play_style_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]


In [14]:
# 遍历所有游戏数据
for idx, game in enumerate(games):
    num  = 8
    moves_list = game.split(',')
    # 计算当前游戏的长度
    current_length = len(moves_list)
    # 计算需要添加的空移动数量
    if current_length < num:
        padding_length = (num - (current_length % num)) %num
        # 获取第一个动作作为padding_value
        padding_value = moves_list[0]
        # 使用列表切片添加空移动
        moves_list = [padding_value] * padding_length + moves_list
        # 更新games中的数据
        games[idx] = ','.join(moves_list)
        

In [15]:
x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_Zero(moves_list))

x_testing = np.array(x_testing)
x_testing.shape


predictions = model.predict(x_testing)
prediction_numbers = np.argmax(predictions, axis=1)

with open('./public_private_submission_template.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

313/313 [==============================] - 13s 41ms/step
